## Download new modules

```bash
python -m spacy download de_core_news_md
```

In [16]:
%matplotlib inline
### Standard imports
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from importlib import reload  

### HEPKIT imports
import hepkit.hepData as hd
import hepkit.hepPlot as hp
import hepkit.hepFormat as hf
### Settings
hf.DefaultSetup()
hf.DefaultSetup()

# Necessary imports
import support as sp
from googletrans import Translator as T1
from translate import Translator as T2
from deep_translator import GoogleTranslator as T3
import spacy
import time

In [30]:
### TEST TRANSLATOR
# To make sure it's working
phrase = "Nel mezzo del cammin di nostra vita, mi ritrovai per una selva oscura, che la dritta via era smarrita."
from_lang = 'it'
to_lang = 'en'

# GOOGLETRANS
# translator = T1()
# translator.translate(phrase,src=from_lang,dest=to_lang).text

## TRANSLATE
# providers = ['mymemory','microsoft']
# provider = providers[0]
# translator = T2(provider=provider,from_lang=from_lang,to_lang=to_lang)
# translator.translate(phrase)

## DEEP_TRANSLATOR
translator = T3(source=from_lang, target=to_lang)
translator.translate(phrase)

'In the middle of the journey of our life, I found myself in a dark forest, that the straight path was lost.'

In [36]:
# Settings
dirIn = '/Users/sdporzio/Software/EbookTranslation/interm_txt'
dirOut = '/Users/sdporzio/Software/EbookTranslation/interm_txt'
filename = 'strange'

# nlp_model = 'pl_core_news_md'
nlp_model = 'de_core_news_md'

# src, dest = 'pl', 'en'
src, dest = 'de', 'en'
# translator = T1()
# translator = T2(provider=provider,from_lang=from_lang,to_lang=to_lang)
translator = T3(source=src, target=dest)

maxCharacters = 300

In [37]:
reload(sp)
# Load model
nlp = spacy.load(nlp_model)

fin = dirIn+'/'+filename+'.txt'
fout = dirOut+'/'+filename+'_bilang.txt'

# Load text
with open(fin,'rb') as fileIn:
    text_or = fileIn.read()
# Decode it
text = text_or.decode(encoding='utf8')
# OPTIONAL: convert tabs to spaces and unencodable /xad's
# text = text.replace('\t',' ')
text = text.replace('\xad','')
# Remove stupid empty lines and split in paragraphs
text = sp.RemoveLines(text)
pars = text.split('\n\n')
# Split long paragraphs in shorter ones
new_pars = []
for p in tqdm(pars):
    shorter_p = sp.RebinPhrases(nlp,p,maxCharacters)
    for shp in shorter_p:
        if shp!='': new_pars.append(shp)

In [39]:
reload(sp)
### PARAMETERS
# Number of phrases in paragraph to convert
nSent = 15
# Number of seconds to wait between each translation
sleepTime = 0.25


# Compose text with translation
out_text = ''
lims = sp.GetDivisions(new_pars,nSent)
for lim in tqdm(lims):
    start = lim[0]
    end = lim[1]
    new_block = new_pars[start:end]
    new_merged_block = '\n'.join(new_block)
    new_tblock = new_merged_block
    attempts = 0
    while (new_tblock == new_merged_block and attempts<20) :
#         new_tblock = translator.translate(new_merged_block,src=src,dest=dest).text # T1
#         new_tblock = translator.translate(new_merged_block,src=src,dest=dest).text # T2
        new_tblock = translator.translate(new_merged_block) # T3
        time.sleep(sleepTime)
        attempts+=1
    if attempts>=20:
        print("Gave up at:")
        print(new_merged_block)
        break
    split_tblock = new_tblock.split('\n')
    for i in range(len(new_block)):
        out_text += new_block[i]
        out_text += '\n\n'
        out_text += split_tblock[i]
        out_text += '\n\n\n'
        
# Save out to text
with open(fout,'w') as fileOut:
    text = fileOut.write(out_text)
    
hd.SlackMe("Ebook converted.")